In [51]:
!pip install biopython==1.77

     |████████████████████████████████| 2.3 MB 1.2 MB/s eta 0:00:01
  Attempting uninstall: biopython
    Found existing installation: biopython 1.78
    Uninstalling biopython-1.78:
      Successfully uninstalled biopython-1.78


In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict

In [2]:
import random

In [3]:
import pandas as pd

In [4]:
# read names and postions from narrowPeak file
positions = defaultdict(list)
with open('curax(14h)-Z22-vs-IgG-[FLAG-ZBP1-PLUS].narrowPeak') as f:
    for line in f:
        chrom, start, stop, name, score, strand, signalValue, pValue, qValue, peak = line.split()
        positions[chrom].append((int(start), int(stop), int(score), float(signalValue), float(pValue), float(qValue), int(peak), '14h'))
with open('curax(10h-14h)-Z22-vs-IgG-[FLAG-ZBP1-MINUS].narrowPeak') as f:
    for line in f:
        chrom, start, stop, name, score, strand, signalValue, pValue, qValue, peak = line.split()
        positions[chrom].append((int(start), int(stop), int(score), float(signalValue), float(pValue), float(qValue), int(peak), '10h-14h'))        

In [5]:
# search for short sequences
short_seq_records = []
f = open('train_data.csv', 'w') 
for chrom in positions:
    print(chrom)
    long_seq_record = SeqIO.to_dict(SeqIO.parse(open(chrom + '.fa'), 'fasta'))[chrom]
#     print(long_seq_record)
    for (start, stop, score, signalValue, pValue, qValue, peak, file) in positions[chrom]:
        long_seq = long_seq_record.seq
        alphabet = long_seq.alphabet
        short_seq = str(long_seq)[start-1:stop]
        short_seq_record = SeqRecord(Seq(short_seq, alphabet), id=chrom, description='')
        short_seq_records.append(short_seq_record)
#         print(short_seq)
        with open('train_data.csv', 'a') as f:
            f.write(','.join([chrom, str(score), str(signalValue), str(pValue), str(qValue), str(peak), file, short_seq]))
            f.write('\n')


chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr1_GL456211_random
chr1_GL456221_random
chr2
chr3
chr4
chr5
chr5_GL456354_random
chr6
chr7
chr8
chr9
chrM
chrUn_GL456370
chrUn_GL456383
chrUn_GL456390
chrUn_GL456392
chrUn_GL456393
chrUn_GL456396
chrUn_JH584304
chrX
chrY
chr4_GL456216_random
chr4_JH584293_random
chr4_JH584294_random
chrUn_GL456378
chrUn_GL456389


In [6]:
file1 = open("train_data.csv")
df = pd.read_csv('train_data.csv', delimiter=',',
                names=['chrom', 'score', 'signalValue', 'pValue', 'qValue', 'peak', 'file', 'short_seq'])
df.head(10)

,chrom,score,signalValue,pValue,qValue,peak,file,short_seq
0,chr1,84,4.19611,12.50863,8.41826,176,14h,cctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcc...
1,chr1,29,4.12410,6.59635,2.90590,139,14h,CTCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCA...
2,chr1,107,2.47923,14.95097,10.76389,494,14h,gcaaatctccaccaaattggattcacctgtatcctggtacacctgc...
3,chr1,86,5.25893,12.75605,8.65332,231,14h,agaacacaaatgccagcccaggctactatacccggccaaactctca...
4,chr1,82,4.93616,12.34533,8.26059,224,14h,acggtcacccgtgcagcctgccctccgcggagtcccggagccaaga...
5,chr1,105,6.77531,14.68437,10.50520,207,14h,tgaagatcagccttagtgcatggtgatctgataggatacatgggac...
6,chr1,233,7.36020,27.82114,23.39590,171,14h,gttctctggcgcaccctctcacctgttcagactaatttcctaagtt...
7,chr1,53,4.20488,9.30839,5.39266,202,14h,taaagaattacaggagagcactgctaaagagttacaggcccttaaa...
8,chr1,128,5.22320,17.05809,12.80588,196,14h,tcccgtggggagtcccgtgtgggcccttgcgggtgttgggcaagac...
9,chr1,68,3.95599,10.90123,6.88675,41,14h,ctaagccacagcagcagcggtcgccatcttggtccgggacccgccg...


In [7]:
#generating lengths for negative class  
import numpy as np
from sklearn.neighbors import KernelDensity
X = []
for chrom in positions:
    for (start, stop, score, signalValue, pValue, qValue, peak, file) in positions[chrom]:
        X.append(stop - start + 1)

Y = np.array(X).reshape(-1, 1)
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(Y)
for i in kde.sample(n_samples=10):
    print(int(i[0]))
print(int(kde.sample(n_samples=1)[0])) #sample length

585
268
401
414
578
289
314
746
2144
533
660


In [9]:
positions_neg = defaultdict(list)
for chrom in positions:
    long_seq_record = SeqIO.to_dict(SeqIO.parse(open(chrom + '.fa'), 'fasta'))[chrom]
    print(chrom)
    for i in range(len(long_seq_record.seq)//10000):
        begin = random.randint(0,len(long_seq_record.seq))
        length = int(kde.sample(n_samples=1)[0]) #sample length
        end = begin + length - 1
        if end > len(long_seq_record.seq):
            continue
        for (start, stop, score, signalValue, pValue, qValue, peak, file) in positions[chrom]:
            if ((begin >= start and end <= stop) 
            or (begin < start and end >= start and end <= stop) 
            or (begin >= start and begin <= stop and end > stop)):
                continue
        positions_neg[chrom].append((begin, end))

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr1_GL456211_random
chr1_GL456221_random
chr2
chr3
chr4
chr5
chr5_GL456354_random
chr6
chr7
chr8
chr9
chrM
chrUn_GL456370
chrUn_GL456383
chrUn_GL456390
chrUn_GL456392
chrUn_GL456393
chrUn_GL456396
chrUn_JH584304
chrX
chrY
chr4_GL456216_random
chr4_JH584293_random
chr4_JH584294_random
chrUn_GL456378
chrUn_GL456389


In [13]:
for chrom in positions:
    print(len(positions[chrom])) #distribution check

186
242
233
166
193
188
247
168
268
124
216
1
2
346
222
334
336
1
256
275
320
203
3
2
17
10
7
6
8
40
227
51
1
1
2
1
6


In [10]:
for chrom in positions_neg:
    print(len(positions_neg[chrom])) #distribution check

19547
13069
12208
12012
12042
12490
10404
9820
9498
9070
6143
24
20
18211
16003
15649
15182
19
14973
14544
12939
12459
1
2
3
1
2
5
2
11
17102
9174
6
20
19
3
2


In [11]:
short_seq_records = []

f = open('train_data_neg.csv', 'w') 
for chrom in positions:
    print(chrom)
    long_seq_record = SeqIO.to_dict(SeqIO.parse(open(chrom + '.fa'), 'fasta'))[chrom]
    for (begin, end) in positions_neg[chrom]:
        long_seq = long_seq_record.seq
        alphabet = long_seq.alphabet
        short_seq = str(long_seq)[begin-1:end]
        short_seq_record = SeqRecord(Seq(short_seq, alphabet), id=chrom, description='')
        short_seq_records.append(short_seq_record)

#         print(short_seq)
        
        with open('train_data_neg.csv', 'a') as f:
            f.write(chrom + ',' + short_seq + '\n')

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr1_GL456211_random
chr1_GL456221_random
chr2
chr3
chr4
chr5
chr5_GL456354_random
chr6
chr7
chr8
chr9
chrM
chrUn_GL456370
chrUn_GL456383
chrUn_GL456390
chrUn_GL456392
chrUn_GL456393
chrUn_GL456396
chrUn_JH584304
chrX
chrY
chr4_GL456216_random
chr4_JH584293_random
chr4_JH584294_random
chrUn_GL456378
chrUn_GL456389


In [12]:
file2 = open("train_data_neg.csv")
df_neg = pd.read_csv('train_data_neg.csv', delimiter=',',
                names=['chrom', 'short_seq'])
df_neg.head(10)

,chrom,short_seq
0,chr1,gtttcaggcatccattaagatcttgagaactgtgtcctgaagataa...
1,chr1,ATATTATACTAATATTACAATACAAAGAGAAgggtatatatctcag...
2,chr1,ttttttttttttttggaggcactcaaagctatgattttccctctta...
3,chr1,TAATCCCTCCTGGAATTGGGAACATTGAAGGCTATTCCTCTGCCTT...
4,chr1,ACAAATACTGTCATCCTATGGTACTTATGTTACACAGAAGAACAGC...
5,chr1,TTGTTTCTGTGCTCCACGGATTCCTAAGAGCTGAGGGGTATTCACC...
6,chr1,ATAACACATACCATGGTTAAAAGTAAAGATAATATTTAAATTCTGC...
7,chr1,CACCAGGAAACTGCACAGTCCTGGGCTAGCGTCATGGGGTTCTGAG...
8,chr1,ATTTGATTAGTAAATTTATGATTCCTGTGCAACTTTACTTCAGTTT...
9,chr1,atcctatctgtatttagctttctttgtccttttgttatgtataatt...
